# 1. Import Data

## Import dependencies, set up environment

1. `python3 -m venv .venv`
2. utilize virtual environment
    - (LINUX/MAC) `source .venv/bin/activate`
    - (WINDOWS) `.venv\Scripts\Activate.    ps1`
3. `cd project_folder/util`
4. `pip install -r requirements.txt`

In [28]:
# Jupyter magic
%run ../util/dependencies.py

## FAA Service Discrepancy Reports

In [ ]:
filepath = '../data/01_raw'
filenames = os.listdir(filepath)
filenames = [filepath + '/' + f for f in filenames]

# try opening first file
if(not pd.read_html(filenames[0])):
   print('ERROR LOADING FILES')

df = pd.read_html(filenames[0])[0]

# concat other files
if len(filenames) > 1:
    for f in filenames[1:]:
        df = pd.concat([df, pd.read_html(f)[0]], ignore_index=True)

['../data/01_raw/202503.xls', '../data/01_raw/202501.xls', '../data/01_raw/202502.xls']
['../data/01_raw/202501.xls', '../data/01_raw/202502.xls']
<class 'pandas.DataFrame'>
RangeIndex: 3258 entries, 0 to 3257
Data columns (total 76 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   OperatorControlNumber    3258 non-null   str    
 1   DifficultyDate           3258 non-null   str    
 2   SubmissionDate           3258 non-null   str    
 3   OperatorDesignator       3247 non-null   str    
 4   SubmitterDesignator      3247 non-null   str    
 5   SubmitterTypeCode        3258 non-null   str    
 6   ReceivingRegionCode      3258 non-null   str    
 7   ReceivingDistrictOffice  3258 non-null   int64  
 8   SDRType                  3258 non-null   str    
 9   JASCCode                 3258 non-null   int64  
 10  NatureOfConditionA       3258 non-null   str    
 11  NatureOfConditionB       70 non-null     str    
 

In [38]:
print(df.isnull().sum())

df.to_csv("../data/02_csv/SDR_COMPOSITE_EXPORT.csv", index=False)

OperatorControlNumber       0
DifficultyDate              0
SubmissionDate              0
OperatorDesignator         11
SubmitterDesignator        11
                         ... 
CrackLength              3150
NumberOfCracks           3087
CorrosionLevel           2714
StructuralOther          3219
Discrepancy                 0
Length: 76, dtype: int64
